In [1]:
import json  
import random  
import numpy as np  
from sklearn.preprocessing import LabelEncoder  
from tensorflow.keras.preprocessing.text import Tokenizer  
from tensorflow.keras.preprocessing.sequence import pad_sequences  
from tensorflow.keras.models import Sequential  
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense  
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint 
from tensorflow.keras.layers import SpatialDropout1D, LSTM, Bidirectional, Dropout

In [2]:
# Load the data
with open('Intent.json') as file:
    data = json.load(file)

In [3]:
# Lists of texts and labels
texts = []  
labels = [] 
for intent in data['intents']:
    for text in intent['text']:
        texts.append(text)       
        labels.append(intent['intent'])  
# I got the structure of code from lecture notes, but I changed the code  

In [4]:
print(texts[:5])    
print(labels[:5])  

['Hi', 'Hi there', 'Hola', 'Hello', 'Hello there']
['Greeting', 'Greeting', 'Greeting', 'Greeting', 'Greeting']


In [5]:
print('Number of phrases:', len(texts)) 
print('Number of intents:', len(labels))   

Number of phrases: 143
Number of intents: 143


In [6]:
# Encode labels to integers
encoder = LabelEncoder()
encoder.fit(labels)
training_labels_encoded = encoder.transform(labels) 
num_classes = len(encoder.classes_) 
# let's check, how many unique labels I have  
print('Number of unique labels:', num_classes)  

Number of unique labels: 22


In [7]:
# Tokenize the texts    
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(texts)
VOCAB_SIZE = len(tokenizer.word_index)+1
print('vocab size:', VOCAB_SIZE)

vocab size: 117


In [8]:
# convert texts to sequences
text_seq = tokenizer.texts_to_sequences(texts)
max_len = max(len(seq) for seq in text_seq)

In [9]:
# Pad sequences to the same length  
padded_sequences = pad_sequences(text_seq, maxlen=max_len, truncating='post')

In [10]:
model = Sequential([
    # Embedding layer
    # I have VOCAB_SIZE words in my vocabulary, and I want to represent each word with a 32 dimensional vector.
    Embedding(VOCAB_SIZE, 32, input_length=max_len),
    SpatialDropout1D(0.2),
    # Here I use a Bidirectional LSTM layer with 64 units.
    # The return_sequences parameter is set to False, which means that the output of the LSTM layer will be a 2D tensor of shape
    Bidirectional(LSTM(64, return_sequences=False, dropout=0.2, recurrent_dropout=0.2)),
    # A fully connected layer with 128 neurons.
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])


c:\Users\murta\Desktop\ML&DE\Fourth_Semester\Advanced_ML\.venv\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [11]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [12]:
# When I ran the model, I noticed that the model was overfitting,
# that is why I added the EarlyStopping and ModelCheckpoint callbacks.
# Monitor the validation loss and stop training if it doesn't improve for 3 epochs.
# Save the best model based on validation loss.
#early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
#checkpoint = ModelCheckpoint('best_chatbot.h5', monitor='val_loss', save_best_only=True)

In [13]:
history = model.fit(padded_sequences,np.array(training_labels_encoded),
    epochs=100,
    batch_size=64,
    #callbacks=[early_stop, checkpoint],
    verbose=1
)

Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.0292 - loss: 3.0906
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.0569 - loss: 3.0901
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.0588 - loss: 3.0855
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.0662 - loss: 3.0853 
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0775 - loss: 3.0807
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.0756 - loss: 3.0793
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0869 - loss: 3.0747
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.1161 - loss: 3.0694
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0619 - loss: 3.0715
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.1071 - loss: 3.0639
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.0849 - loss: 3.0480
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.1274 - l

In [14]:
#model.load_weights('best_chatbot.h5')

In [15]:
# to get a response using the trained model
def get_response(user_input):
    # tokenize and pad the input
    seq = tokenizer.texts_to_sequences([user_input])
    pad = pad_sequences(seq, maxlen=max_len, truncating='post')
    # predict intent
    pred = model.predict(pad)
    intent_index = np.argmax(pred)
    intent_label = encoder.inverse_transform([intent_index])[0]
    # select a random response
    for intent in data['intents']:
        if intent['intent'] == intent_label:
            return random.choice(intent['responses'])
    return "I'm sorry, I don't understand."

In [16]:
def chat():
    print("Chatbot: Hello! Type 'quit' to exit.")
    while True:
        user_input = input('You: ')
        if user_input.strip().lower() == 'quit':
            print('Chatbot: Goodbye!')
            break
        print(f'Chatbot: {get_response(user_input)}')

chat()

Chatbot: Hello! Type 'quit' to exit.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 440ms/step
Chatbot: Hola human, please tell me your GeniSys user
Chatbot: Goodbye!


In [ ]:
# Since my dataset is small, the model is overfitting. 